In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras.layers as L
from tensorflow.keras import Model
from sklearn.metrics import f1_score
from tensorflow.keras import callbacks
from tensorflow.keras.optimizers import Adam 

# Load and process data

In [2]:
train = pd.read_csv("../input/liverpool-ion-switching/train.csv")
test = pd.read_csv("../input/liverpool-ion-switching/test.csv")
sub = pd.read_csv("../input/liverpool-ion-switching/sample_submission.csv", dtype=dict(time=str))

In [3]:
n_classes = train.open_channels.unique().shape[0]

In [4]:
seq_len = 1000

X = train.signal.values.reshape(-1, seq_len, 1)
y = train.open_channels.values.reshape(-1, seq_len, 1)

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

X_test = test.signal.values.reshape(-1, seq_len, 1)

# Modelling

In [5]:
def build_model(n_classes, seq_ln = 500, n_units = 256, learning_rate = 0.007):
    inputs = L.Input(shape = (seq_len, 1))
    x = L.Dense(n_units, activation = 'linear')(inputs)
    
    x = L.Bidirectional(L.LSTM(n_units, return_sequences = True))(x)
#     x = L.Dense(n_units, activation = 'relu')(x)
    x = L.Bidirectional(L.LSTM(n_units, return_sequences = True))(x)
#     x = L.Dense(n_units, activation = 'relu')(x)
    x = L.Bidirectional(L.LSTM(n_units, return_sequences = True))(x)
    x = L.Dense(n_units, activation = 'relu')(x)
#     x = L.Bidirectional(L.LSTM(n_units, return_sequences = True))(x)
#     x = L.Dense(n_units, activation = 'relu')(x)
#     x = L.Bidirectional(L.LSTM(n_units, return_sequences = True))(x)
    
#     x = L.Dense(n_units, activation = 'relu')(x)
    x = L.Dense(n_units, activation = 'relu')(x)
    x = L.Dense(n_classes, activation = 'softmax')(x)
    
    model = Model(inputs = inputs, outputs = x)
    model.compile(optimizer=Adam(lr=learning_rate), loss = 'sparse_categorical_crossentropy')
    
    return model
   
# learning_rate = 0.007
def lrs(epoch, learning_rate = 0.007):
    if epoch < 50:
        lr = learning_rate
    elif epoch < 100:
        lr = learning_rate/10
    else:
        lr = learning_rate/100
        
    return lr

In [6]:

model = build_model(n_classes, seq_len, learning_rate = 0.007)
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 1000, 1)]         0         
_________________________________________________________________
dense (Dense)                (None, 1000, 256)         512       
_________________________________________________________________
bidirectional (Bidirectional (None, 1000, 512)         1050624   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1000, 512)         1574912   
_________________________________________________________________
bidirectional_2 (Bidirection (None, 1000, 512)         1574912   
_________________________________________________________________
dense_1 (Dense)              (None, 1000, 256)         131328    
_________________________________________________________________
dense_2 (Dense)              (None, 1000, 256)         65792 

In [7]:
class F1Callback(callbacks.Callback):
    def __init__(self, X_val, y_val):
        super().__init__()
        self.X = X_val
        self.y = y_val.reshape(-1)
    def on_epoch_begin(self, epoch, logs=None):
        if epoch == 0:
            return
        pred = (
            model
            .predict(self.X, batch_size=64)
            .argmax(axis=-1)
            .reshape(-1)
        )
        
        score = f1_score(self.y, pred, average='macro')
        
        print(f"val_f1_macro: {score:.4f}")

In [8]:
lr_schedule = callbacks.LearningRateScheduler(lrs)

model.fit(
    X_train, y_train, 
    batch_size=64,
    epochs=200,
    callbacks=[
        callbacks.ReduceLROnPlateau(),
        F1Callback(X_valid, y_valid),
        callbacks.ModelCheckpoint('model.h5'),
        lr_schedule
    ],
    validation_data=(X_valid, y_valid)
)

Train on 4000 samples, validate on 1000 samples
Epoch 1/200
4000/4000 [==============================] - 58s 15ms/sample - loss: 1.5710 - val_loss: 1.2838
val_f1_macro: 0.2972
Epoch 2/200
4000/4000 [==============================] - 46s 12ms/sample - loss: 1.2078 - val_loss: 1.0302
val_f1_macro: 0.4092
Epoch 3/200
4000/4000 [==============================] - 46s 12ms/sample - loss: 1.0418 - val_loss: 0.8791
val_f1_macro: 0.4640
Epoch 4/200
4000/4000 [==============================] - 46s 12ms/sample - loss: 0.8246 - val_loss: 0.7788
val_f1_macro: 0.5263
Epoch 5/200
4000/4000 [==============================] - 46s 12ms/sample - loss: 0.7329 - val_loss: 0.6326
val_f1_macro: 0.5928
Epoch 6/200
4000/4000 [==============================] - 46s 12ms/sample - loss: 0.6369 - val_loss: 0.6245
val_f1_macro: 0.6263
Epoch 7/200
4000/4000 [==============================] - 46s 12ms/sample - loss: 0.6223 - val_loss: 0.6451
val_f1_macro: 0.6308
Epoch 8/200
4000/4000 [==============================] -

# Evaluate

In [9]:
model.load_weights('model.h5')
valid_pred = model.predict(X_valid, batch_size=64).argmax(axis=-1)
f1_score(y_valid.reshape(-1), valid_pred.reshape(-1), average='macro')

0.882512961138132

# Submit

In [10]:
test_pred = model.predict(X_test, batch_size=64).argmax(axis=-1)
sub.open_channels = test_pred.reshape(-1)
sub.to_csv('submission.csv', index=False)